In [1]:
import os
import pylab  as pl
import pandas  as pd
import numpy  as np
import math
from datetime import datetime, timedelta, timezone

In [2]:
def get_mean(a):
    number=a.size
    down=math.ceil(number*0.25)
    top=math.floor(number*0.75)
    #print(number)
    #print(number)
    #print(top)
    if number> 50:
        average=a[down-1:top-1].mean()
        return average

In [3]:
def time_mean(a):
    b=pd.to_datetime(a).dropna().astype(np.int64)
    number=a.size
    down=math.ceil(number*0.25)
    top=math.floor(number*0.75)
    #print(number)
    #print(number)
    #print(top)
    if number> 50:
        average=b[down-1:top-1].mean()
        time_ave=pd.to_datetime(average,format='%Y-%m-%d %H:%M:%S')
        time_ave=datetime.strftime(time_ave,"%Y-%m-%d %H:%M:%S")
        return time_ave

In [4]:
def search(s, path=os.path.abspath('.')):  #os.path.abspath(path)：绝对路径

    for z in os.listdir(path):
        if os.path.isdir(path + os.path.sep + z):  # os.path.sep:路径分隔符 linux下就用这个了’/’
            #print('Currnet:', path)
            path2 = os.path.join(path, z) #；os.path.join(): 常用来链接路径
            #print('future:', path2)
            search(s, path2)
        elif os.path.isfile(path + os.path.sep + z): #检验给出的路径是否是一个文件：os.path.isfile()来自 <http://blog.csdn.net/devil_2009/article/details/7941241>
            if s in z:
                #print(os.path.join(path, z))
                filenames.append(os.path.join(path, z))
                #with open(path + os.path.sep + z, 'r') as fr:
                #    with open('save.txt', 'a') as fw:
                #        fw.write(path + '\t' + fr.read())

In [6]:
filenames=[]
#OUT = pd.DataFrame(columns=['Stid','Lat','Lon','year', 'month','day','rain','rain208','rain820'])
OUT = pd.DataFrame()
search('ccn_100', '.')
filenames=sorted(filenames)#;防止顺序不对
for filename in filenames:
    data=pd.read_csv(filename,sep=',',skiprows=[0,1,2,3])
    print(filename[15:23])
    data['diff_or'] = (data[" Current SS"].diff().ne(0)).astype(int)
    data['labels'] = data['diff_or'].cumsum()
    data=data.sort_values(by=['    Time'],na_position='first')
# #df=data.groupby(["labels"])[' CCN Number Conc'].agg([get_mean])
    da=data.groupby(["labels"]).agg({' CCN Number Conc':[get_mean],' Current SS':["mean"],'    Time':[time_mean]})
#读取txt数据
    times=pd.to_datetime(filename[15:23],format='%y%m%d%H')
    print(times)
    #da.insert(0,'DATE',times)
    da.insert(0,'Date_BJC',times+timedelta(hours=8))
    #time=pd.to_datetime(data.loc[[a],['year', 'month', 'day']])
    #PRE.insert(0,'Date',a)
    print(da)
    OUT=pd.concat([OUT,da])
OUT.sort_values(by=['Date_BJC'],inplace=True)
#OUT.to_csv('NUIST_CCN_ave1.csv', encoding='utf-8',index=False)
OUT.to_csv('NUIST_CCN_ave2.csv',header=["Date_BJC","CCN_Number_Conc","Current_SS","Time"],float_format='%8.2f',index=True)

13060312
2013-06-03 12:00:00
                  Date_BJC  CCN Number Conc  Current SS                 Time
                                   get_mean        mean            time_mean
labels                                                                      
1      2013-06-03 20:00:00      9278.468824         1.0  2019-04-08 12:00:32
2      2013-06-03 20:00:00      1096.452378         0.1  2019-04-08 12:08:36
3      2013-06-03 20:00:00      4427.102886         0.2  2019-04-08 12:18:37
4      2013-06-03 20:00:00      4550.867600         0.3  2019-04-08 12:23:37
5      2013-06-03 20:00:00      5400.607733         0.4  2019-04-08 12:28:37
6      2013-06-03 20:00:00      5104.093333         0.5  2019-04-08 12:33:37
7      2013-06-03 20:00:00      5450.126600         0.6  2019-04-08 12:38:37
8      2013-06-03 20:00:00      6656.847400         0.7  2019-04-08 12:43:37
9      2013-06-03 20:00:00      9243.822867         0.8  2019-04-08 12:48:37
10     2013-06-03 20:00:00     10087.088400    

In [ ]:
OUT